# Классификация тональности текста

Для определения тональности применим величины TF-IDF как признаки.

Анализ тональности текста, или сентимент-анализ (от англ. sentiment, «настроение»), выявляет эмоционально окрашенные слова. Этот инструмент помогает компаниям оценивать, например, реакцию на запуск нового продукта в интернете. На разбор тысячи отзывов человек потратит несколько часов, а компьютер — пару минут.

Оценить тональность — значит отметить текст как позитивный или негативный. То есть мы решаем задачу классификации, где целевой признак равен «1» для положительного текста и «0» для отрицательного. Признаки — это слова из корпуса и их величины TF-IDF для каждого текста.

### Задача

- Обучите логистическую регрессию так, чтобы она определяла тональность текста.

- Подсчитайте величину TF-IDF для текстов. Лемматизированные тексты твитов для обучения находятся в файле tweets_lemm_train.csv. Целевой признак вы найдёте в столбце positive.

- Обученной моделью классификации определите результаты предсказания для тестовой выборки твитов, которая лежит в файле tweets_lemm_test.csv. В этой выборке целевого признака нет. Сохраните предсказания в столбце positive. Таблицу с результатом сохраните как csv-файл, но чтобы тренажёр принял файл, не указывайте расширение (например, назовите файл 'predictions')

- Значение accuracy вашей модели должно быть не меньше 0.62.
- Для предсказания ответов повторно вычислите величину TF-IDF для вектора с тестовой выборкой. Примените метод transform() к объекту TfidfVectorizer.